# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from patsy import dmatrices, dmatrix, demo_data


In [4]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [6]:
colunas_excluidas = ['Unnamed: 0', 'data_ref', 'id_cliente']
df_filtrado = df.drop(columns=colunas_excluidas)

df_filtrado.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [7]:
y, X = dmatrices('np.log(renda) ~ ' + ' + '.join([f'C({col})' if df_filtrado[col].dtype == 'object' or df_filtrado[col].dtype == bool else col for col in df_filtrado.columns if col != 'renda']), df_filtrado, return_type='dataframe')

model = sm.OLS(y, X)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Mon, 20 May 2024   Prob (F-statistic):               0.00
Time:                        23:49:11   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.5847      0.235     28.006      0.000       6.124       7.046
C(sexo)[T.M]                            0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo)[T.True]             0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel)[T.True]              0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda)[T.Bolsista]               0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda)[T.Empresário]             0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda)[T.Pensionista]           -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda)[T.Servidor público]       0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao)[T.Pós graduação]            0.1071      0.159      0.673      0.501      -0.205       0.419
C(educacao)[T.Secundário]              -0.0141      0.072     -0.196      0.844      -0.155       0.127
C(educacao)[T.Superior completo]        0.0938      0.072      1.298      0.194      -0.048       0.235
C(educacao)[T.Superior incompleto]     -0.0432      0.078     -0.555      0.579      -0.196       0.109
C(estado_civil)[T.Separado]             0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil)[T.Solteiro]             0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil)[T.União]               -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil)[T.Viúvo]                0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia)[T.Casa]             -0.0442      0.054     -0.815      0.415      -0.150       0.062
C(tipo_residencia)[T.Com os pais]      -0.0258      0.060     -0.427      0.670      -0.144       0.093
C(tipo_residencia)[T.Comunitário]      -0.1196      0.105     -1.135      0.256      -0.326       0.087
C(tipo_residencia)[T.Estúdio]           0.0665      0.099      0.670      0.503      -0.128       0.261
C(tipo_residencia)[T.Governamental]    -0.0571      0.066     -0.865      0.387      -0.186       0.072
qtd_filhos                             -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                   0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego                           0.0617      0.001     59.398      0.000       0.060       0.064
qt_pessoas_residencia                   0.2926      0.108      2.716      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.858   Durbin-Watson:                   2.023
Prob(Omnibus)

In [9]:
# Ajustando a fórmula excluindo manualmente a variável menos significativa "tipo_residencia"
new_formula_sanitized_no_residencia = 'np.log(renda) ~ ' + ' + '.join([f'C({col})' if df_filtrado[col].dtype == 'object' or df_filtrado[col].dtype == bool else col for col in df_filtrado.columns if col != 'renda' and col != 'tipo_residencia'])

# Recriando as matrizes de design e ajustar o modelo novamente
y, X = dmatrices(new_formula_sanitized_no_residencia, df_filtrado, return_type='dataframe')
new_model_sanitized_no_residencia = sm.OLS(y, X)
new_results_sanitized_no_residencia = new_model_sanitized_no_residencia.fit()

new_results_sanitized_no_residencia.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Mon, 20 May 2024   Prob (F-statistic):               0.00
Time:                        23:49:17   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              6.5411      0.229     28.532      0.000       6.092       6.991
C(sexo)[T.M]                           0.7893      0.015     53.999      0.000       0.761       0.818
C(posse_de_veiculo)[T.True]            0.0442      0.014      3.125      0.002       0.016       0.072
C(posse_de_imovel)[T.True]             0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda)[T.Bolsista]              0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda)[T.Empresário]            0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda)[T.Pensionista]          -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda)[T.Servidor público]      0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao)[T.Pós graduação]           0.1104      0.159      0.695      0.487      -0.201       0.422
C(educacao)[T.Secundário]             -0.0097      0.072     -0.135      0.893      -0.150       0.131
C(educacao)[T.Superior completo]       0.0980      0.072      1.359      0.174      -0.043       0.239
C(educacao)[T.Superior incompleto]    -0.0384      0.078     -0.495      0.621      -0.191       0.114
C(estado_civil)[T.Separado]            0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil)[T.Solteiro]            0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil)[T.União]              -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil)[T.Viúvo]               0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                            -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                  0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                          0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                  0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.809
Skew:                           0.019   Prob(JB):                        0.667
Kurtosis:                       3.009   Cond. No.                     2.16e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Pode-se notar que comparando o modelo incial com o modelo final, se manteve o mesmo valor de R², logo ja que sabemos que o ultimo
modelo foi feito com a remoção de variáveis não significativas e mesmo assim manteve o mesmo valor de R² podemos dizer que ele tem uma
melhor performa.